# Predict patient conditions

* Use current patient conditions to predict which conditions they would have in the future
* Data source: Synthea (synthetic open source) or Optum (proprietary) claims data (this code is able to handle both)
* Use case: adding medications to the feature vector we could measure medication effects

# Research agenda


* Mostly centered around applications of Variational Auto-Encoders (VAE)
* Main issue: what is a good measure of performance for a VAE?
* Solution: use encoding in some downstream task.
* How this project fits in: it's a first pass at the downstream task.

In [445]:
!pip install pyTigerGraph

## Connect to a Synthea TigerGraph server


In [2]:
import getpass

server = 'https://yaniv.i.tgcloud.io'
password = getpass.getpass()

········


In [442]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(
    host=server, 
    graphname="synthea", 
    username="tigergraph",
    password=password,
)

shell = tg.Gsql(conn, client_version="2.6.0")

In [443]:
!ls -l $shell.jarLocation

total 1432
-rw-r--r--  1 ybenami  ybenami  671144 Jul 23 11:27 gsql_client.jar
-rw-r--r--  1 ybenami  ybenami    8518 Jul 23 11:27 my-cert.txt


In [5]:
#!rm -rf $shell.jarLocation

In [6]:
secret = shell.createSecret()
conn.getToken(secret=secret)

('qjv4m187po3f21abtoki1u94d5bemeh0', 1597950428, '2020-08-20 19:07:08')

In [7]:
query = '''
drop query get_all_patients
create query get_all_patients() for graph synthea{
    TYPEDEF TUPLE <description STRING, date DATETIME> CondTup;

    BagAccum <CondTup> @conditions;

    patients = {Patient.*};

    patients = select pat from patients:pat;

    x = select pat from patients:pat-(PATIENT_HAS_CONDITION)-Condition:cond
            accum pat.@conditions += CondTup(cond.description, cond.startDate);

    print patients;
}
install query get_all_patients
'''

print(shell.gsql(query))

Trying version: v2_6_0
Connecting to yaniv.i.tgcloud.io:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The query get_all_patients is dropped.
The query get_all_patients has been added!
Start installing queries, about 1 minute ...
get_all_patients query: curl -X GET 'https://127.0.0.1:9000/query/synthea/get_all_patients'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.

[=================================================================] 100% (1/1) 



In [8]:
query = conn.runInstalledQuery('get_all_patients', sizeLimit=10**10)

In [52]:
len(query[0]['patients'])

109321

In [88]:
%%writefile getFeatures.py

import numpy as np
import pandas as pd
from datetime import datetime
import math

def yaniv_to_ed_query(query):

    query[0]['people'] = query[0].pop('patients')

    for patient in query[0]['people']:
        patient['attributes']['people.dateOfBirth'] = patient['attributes'].pop('birth')
        patient['attributes']['people.dateOfDeath'] = patient['attributes'].pop('death')
        patient['attributes']['people.@gender'] = [(
            'F' if patient['attributes'].pop('gender') == 'female'
            else 'M')]
        patient['attributes']['people.@diagData'] = patient['attributes'].pop('@conditions')
        for condition in patient['attributes']['people.@diagData']:
            condition['diagnosisDate'] = condition.pop('date')
            condition['diagnosis'] = condition.pop('description')
        patient['attributes'].pop('name')
        patient['attributes'].pop('patient_id')
        
    return query


def get_conditions(query, startDate='1900-01-01', endDate='2019-12-31'):
        
    startDate = datetime.strptime(startDate, '%Y-%m-%d')
    endDate = datetime.strptime(endDate, '%Y-%m-%d')

    conditions = [
        condition['diagnosis']
        for patient in query[0]['people'] 
        for condition in patient['attributes']['people.@diagData']
        if (
            datetime.strptime(condition['diagnosisDate'], '%Y-%m-%d %H:%M:%S') 
            >= startDate
            and datetime.strptime(condition['diagnosisDate'], '%Y-%m-%d %H:%M:%S')
            <= endDate
        )
    ]

    conditions = pd.Series(conditions).value_counts()    

    return conditions


def get_live_patients(query, startDate='1900-01-01', endDate='2019-12-31'):

    startDate = datetime.strptime(startDate, '%Y-%m-%d')
    endDate = datetime.strptime(endDate, '%Y-%m-%d')

    patients = [
        patient['v_id'] 
        for patient in query[0]['people']
        if (
            datetime.strptime(
                patient['attributes']['people.dateOfBirth'], '%Y-%m-%d %H:%M:%S') 
            <= endDate
            and datetime.strptime(
                patient['attributes']['people.dateOfDeath'], '%Y-%m-%d %H:%M:%S')
            >= startDate
        )
    ]

    return patients

def make_age_groups(years = 5, top_year = 100):
    age_groups_ranges = [(i,min(i+years-1,top_year)) 
        for i in range(0, top_year, years)] + [(top_year, 140)]

    age_group_titles = [
        'Age {}-{}'.format(start, end) for start, end in age_groups_ranges]

    return age_groups_ranges, age_group_titles


def get_feature_vec(query, conditions, startDate, endDate, age_groups):

    startDate = datetime.strptime(startDate, '%Y-%m-%d')
    endDate = datetime.strptime(endDate, '%Y-%m-%d')

    demog_df = pd.DataFrame([patient['attributes'] 
                             for patient in query[0]['people']])

    demog_df.index = [
        patient['v_id'] for patient in query[0]['people']
    ]

    demog_df = demog_df[[
        'people.@gender',
        'people.dateOfBirth',
        'people.dateOfDeath',
    ]]

    df = pd.DataFrame(
            np.zeros((len(demog_df.index), len(conditions.index))),
            index=demog_df.index, 
            columns=conditions.index,
        )

    for patient in query[0]['people']:

        patient_conditions = [
            condition['diagnosis'] 
            for condition in patient['attributes']['people.@diagData']
            if (
                datetime.strptime(condition['diagnosisDate'], '%Y-%m-%d %H:%M:%S') 
                >= startDate
                and datetime.strptime(condition['diagnosisDate'], '%Y-%m-%d %H:%M:%S')
                <= endDate
                and condition['diagnosis'] in conditions
            )
        ]

        df.loc[patient['v_id'], patient_conditions ] = 1

    return concat_features(
        conditions_df=df, 
        demog_df=demog_df, 
        date=endDate, 
        age_groups=age_groups,
    )

def concat_features(conditions_df, demog_df, date, age_groups):

    dead_df = deceased(demog_df, date)
    gender_df = gender(demog_df)
    age_df = age_group_df(
        df = demog_df, 
        date_for_age = date,
        age_groups=age_groups,
    )
    
    return pd.concat([gender_df, dead_df, age_df, conditions_df], axis=1)


def age_group_df(df, date_for_age, age_groups):

    age_group_df = pd.DataFrame(
        np.zeros((len(df),len(age_groups[0]))),
        index=df.index, 
        columns=age_groups[1]
        )

    for i in df.index:
        age = math.floor(
            (date_for_age
            - datetime.strptime(df.loc[i,'people.dateOfBirth'], '%Y-%m-%d %H:%M:%S')
            ).days/365.25
        )

        for j, age_group in enumerate(age_groups[0]):
            if age >= age_group[0] and age <= age_group[1]:
                age_group_df.loc[i].iloc[j]=1

    return age_group_df


def deceased(df, date):

    dead = df['people.dateOfDeath'].apply(
        lambda x: 1.0 if (
            date - datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        ).days > 0 else 0.0
    )

    dead.name = 'Deceased'

    return dead


def gender(df):

    gender = df['people.@gender'].apply(
        lambda x: 1.0 if x[0]=='F' else 0.0
    )

    gender.name = 'Female'

    return gender

Overwriting getFeatures.py


In [9]:
import getFeatures
import importlib

importlib.reload(getFeatures)

<module 'getFeatures' from '/Users/ybenami/EasyAsPie.ai/MedGraphML/getFeatures.py'>

In [10]:
query = getFeatures.yaniv_to_ed_query(query)

In [51]:
%%writefile feature_weighted_mse.py
import tensorflow as tf

def make_feature_weighted_mse(feature_weights):
    
    feature_weights = tf.reshape(tf.cast(feature_weights, 'float32'), (-1,1))
    
    def feature_weighted_mse(y_true, y_pred):
        
        y_true = tf.cast(y_true, 'float32')
        y_pred = tf.cast(y_pred, 'float32')

        return  `
    
    return feature_weighted_mse

Overwriting feature_weighted_mse.py


## Get the set of all conditions

In [514]:
%%writefile predictConditions.py
import getFeatures
import feature_weighted_mse
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.stats import t    
from statsmodels.stats.multitest import multipletests

def predictConditions(query):
    
    print('Collecting all conditions:\n')
    
    conditions = getFeatures.get_conditions(
        query=query, startDate='2019-01-01', endDate='2020-12-31')
    
    print(conditions)
    

    patients = getFeatures.get_live_patients(
        query=query, startDate='2019-12-31', endDate='2019-12-31')
    
    print('\nNumber of patients', len(patients))
    

    age_groups = getFeatures.make_age_groups()
    
    print('\nAge groups\n', age_groups)
    

    print('\nCompute features: ')
    
    x_df = getFeatures.get_feature_vec(
        query,
        conditions=conditions,
        startDate='2019-01-01', 
        endDate='2019-12-31', 
        age_groups=age_groups)

    print('\nx_df.shape ', x_df.shape)
    
    print('\nCompute labels: ')
    
    y_df = getFeatures.get_feature_vec(
        query,
        conditions=conditions,
        startDate='2020-01-01', 
        endDate='2020-12-31', 
        age_groups=age_groups)
    
    print('\ny_df.shape ', y_df.shape)
    

    train, test = train_test_split(patients, test_size=0.25, random_state=42)
    
    x_train_df = x_df.loc[train]
    y_train_df = y_df.loc[train]
    x_test_df = x_df.loc[test]
    y_test_df = y_df.loc[test]
    
    print('\n\nTrain set:', len(train), 'Test set: ', len(test))
    
    print(
        '\n\nSorted x_train means:\n\n',
        x_train_df.mean().sort_values(ascending=False), 
        '\n\nSorted y_train means:\n\n',
        y_train_df.mean().sort_values(ascending=False)
    )

    filter_below = 20
    print('\nFiltereing conditions with less than {} cases:'.format(filter_below))
    
    x_drop_list = ( 
        set(x_train_df.columns[x_train_df.sum() < filter_below])
        | set(x_test_df.columns[x_train_df.sum() < filter_below])
    )

    x_train_df = x_train_df.drop(x_drop_list, axis=1)
    x_test_df = x_test_df.drop(x_drop_list, axis=1)

    y_drop_list = ( 
        set(y_train_df.columns[y_train_df.sum() < filter_below])
        | set(y_test_df.columns[y_train_df.sum() < filter_below])
    )

    y_train_df = y_train_df.drop(y_drop_list, axis=1)
    y_test_df = y_test_df.drop(y_drop_list, axis=1)

    print(
        '\n\nSorted x_train means:\n\n',
        x_train_df.mean().sort_values(ascending=False), 
        '\n\nSorted y_train means:\n\n\n\n',
        y_train_df.mean().sort_values(ascending=False)
    )
    
    print(
        '\n\nSorted x_test means:\n\n',
        x_test_df.mean().sort_values(ascending=False), 
        '\n\nSorted y_test means:\n\n\n\n',
        y_test_df.mean().sort_values(ascending=False)
    )

    y_weights = 1 / (y_train_df.var() + 1e-3)
    y_weights = y_weights/(y_train_df.var()*y_weights).sum()
    
    print(
        '\n',
        pd.DataFrame(
            [y_train_df.var(), y_weights, y_weights*y_train_df.var()],
             index=['y_train var', 'y_weights', 'var*weight']
        ).transpose()
    )

    wmse = feature_weighted_mse.make_feature_weighted_mse(y_weights)
    
    print(
        '\nBasic benchmark - y means\n', 
        'Train loss',
        wmse(
            y_true=y_train_df.values, 
            y_pred=y_train_df.values.mean(axis=0)
        ).numpy().mean(),
    )

    from sklearn.model_selection import RepeatedKFold

    n_splits = 4
    n_repeats = 2
    alpha=0.00001
    learning_rate=0.001
    patience=30
    
    print('\nTrain linear model using Lasso alpha {} {}-fold CV repeated {} times.\n'.format(
        alpha, n_splits, n_repeats,
    ))
    

    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    
    models=[]
    history=[]
    performance=[]
    
    i=0
    for train_index, validate_index in rkf.split(x_train_df):
        
        i += 1
        print('\n\nFold {} out of {}\n\n'.format(i, n_splits*n_repeats))
        
        x_train, x_validate = x_train_df.iloc[train_index], x_train_df.iloc[validate_index]
        y_train, y_validate = y_train_df.iloc[train_index], y_train_df.iloc[validate_index]
    
        inputs = keras.layers.Input(shape=x_train_df.shape[1])
        outputs = keras.layers.Dense(
            units=y_train_df.shape[1], 
            kernel_regularizer=keras.regularizers.l1(l=alpha),
        )(inputs)
        
        models.append(keras.Model(inputs=inputs, outputs=outputs))

        models[-1].compile(loss=wmse, optimizer=keras.optimizers.Adam(learning_rate=learning_rate))

        history.append(models[-1].fit(
            x=x_train,
            y=y_train,
            batch_size=128,
            epochs=1000,
            validation_data=(x_validate, y_validate),
            callbacks=[
                keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
            ]
        ))
    
        print('\nEvaluate on test set:\n')
        performance.append(models[-1].evaluate(x=x_test_df, y=y_test_df))
        print(performance[-1],'\n')

    print('Test loss mean', np.mean(performance), 'std' , np.std(performance, ddof=1))
    

    
    constant_full = pd.DataFrame(
        np.array([model.layers[1].get_weights()[1] for model in models]).transpose(), 
        index=y_train_df.columns, 
        columns=['Fold {}'.format(i) for i in range(1, 1+n_splits*n_repeats)],
    )
    constant_full.to_csv('constant_full.csv')
    
    coef_mat = np.array([model.layers[1].get_weights()[0] for model in models]).transpose((1, 2, 0))
    
    coef_full = pd.DataFrame(
        [[json.dumps(coef_mat[i,j].tolist()) 
          for j in range(coef_mat.shape[1])] 
         for i in range(coef_mat.shape[0])], 
        columns=y_train_df.columns, 
        index=x_train_df.columns
    ).transpose()
    
    coef_full.to_csv('coef_full.csv')

Overwriting predictConditions.py


In [515]:
import predictConditions
import importlib

importlib.reload(predictConditions)

<module 'predictConditions' from '/Users/ybenami/EasyAsPie.ai/MedGraphML/predictConditions.py'>

In [516]:
predictConditions.predictConditions(query)


Suspected COVID-19                                      81485
COVID-19                                                78851
Fever (finding)                                         72574
Cough (finding)                                         55183
Loss of taste (finding)                                 41275
                                                        ...  
Lupus erythematosus                                         1
Cystic Fibrosis                                             1
Chronic paralysis due to lesion of spinal cord              1
History of amputation of foot (situation)                   1
Blindness due to type 2 diabetes mellitus (disorder)        1
Length: 173, dtype: int64

Number of patients 92258

Age groups
 ([(0, 4), (5, 9), (10, 14), (15, 19), (20, 24), (25, 29), (30, 34), (35, 39), (40, 44), (45, 49), (50, 54), (55, 59), (60, 64), (65, 69), (70, 74), (75, 79), (80, 84), (85, 89), (90, 94), (95, 99), (100, 140)], ['Age 0-4', 'Age 5-9', 'Age 10-14', 'Age 

51894/51894 [==============================] - 1s 15us/sample - loss: 0.8420 - val_loss: 0.8516
Epoch 5/1000
51894/51894 [==============================] - 1s 15us/sample - loss: 0.8404 - val_loss: 0.8511
Epoch 6/1000
51894/51894 [==============================] - 1s 13us/sample - loss: 0.8402 - val_loss: 0.8505
Epoch 7/1000
51894/51894 [==============================] - 1s 23us/sample - loss: 0.8401 - val_loss: 0.8503
Epoch 8/1000
51894/51894 [==============================] - 1s 18us/sample - loss: 0.8400 - val_loss: 0.8504
Epoch 9/1000
51894/51894 [==============================] - 1s 23us/sample - loss: 0.8402 - val_loss: 0.8506
Epoch 10/1000
51894/51894 [==============================] - 1s 15us/sample - loss: 0.8402 - val_loss: 0.8510
Epoch 11/1000
51894/51894 [==============================] - 1s 14us/sample - loss: 0.8402 - val_loss: 0.8506
Epoch 12/1000
51894/51894 [==============================] - 1s 11us/sample - loss: 0.8402 - val_loss: 0.8506
Epoch 13/1000
51894/51894 [==

51894/51894 [==============================] - 1s 15us/sample - loss: 0.8393 - val_loss: 0.8497
Epoch 79/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8394 - val_loss: 0.8497
Epoch 80/1000
51894/51894 [==============================] - 1s 14us/sample - loss: 0.8393 - val_loss: 0.8497
Epoch 81/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8392 - val_loss: 0.8496
Epoch 82/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8393 - val_loss: 0.8497
Epoch 83/1000
51894/51894 [==============================] - 1s 14us/sample - loss: 0.8392 - val_loss: 0.8497
Epoch 84/1000
51894/51894 [==============================] - 1s 14us/sample - loss: 0.8393 - val_loss: 0.8496
Epoch 85/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8392 - val_loss: 0.8493
Epoch 86/1000
51894/51894 [==============================] - 1s 18us/sample - loss: 0.8392 - val_loss: 0.8495
Epoch 87/1000
51894/5189

Epoch 63/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8434 - val_loss: 0.8374
Epoch 64/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8433 - val_loss: 0.8380
Epoch 65/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8434 - val_loss: 0.8379
Epoch 66/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8434 - val_loss: 0.8378
Epoch 67/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8433 - val_loss: 0.8376
Epoch 68/1000
51895/51895 [==============================] - 1s 18us/sample - loss: 0.8433 - val_loss: 0.8380
Epoch 69/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8434 - val_loss: 0.8380
Epoch 70/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8433 - val_loss: 0.8382

Evaluate on test set:

23065/23065 [==============================] - 0s 20us/sample - loss: 0.83770s - loss: 0.837
0.8

51895/51895 [==============================] - 1s 11us/sample - loss: 0.8411 - val_loss: 0.8445
Epoch 66/1000
51895/51895 [==============================] - 1s 10us/sample - loss: 0.8411 - val_loss: 0.8441
Epoch 67/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8410 - val_loss: 0.8442
Epoch 68/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8411 - val_loss: 0.8446
Epoch 69/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8410 - val_loss: 0.8447
Epoch 70/1000
51895/51895 [==============================] - 1s 10us/sample - loss: 0.8411 - val_loss: 0.8437
Epoch 71/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8410 - val_loss: 0.8445
Epoch 72/1000
51895/51895 [==============================] - 1s 10us/sample - loss: 0.8410 - val_loss: 0.8442
Epoch 73/1000
51895/51895 [==============================] - 1s 10us/sample - loss: 0.8411 - val_loss: 0.8445
Epoch 74/1000
51895/5189

...

Epoch 55/1000
51894/51894 [==============================] - 1s 11us/sample - loss: 0.8410 - val_loss: 0.8445
Epoch 56/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8408 - val_loss: 0.8457
Epoch 57/1000
51894/51894 [==============================] - 1s 13us/sample - loss: 0.8409 - val_loss: 0.8450
Epoch 58/1000
51894/51894 [==============================] - 1s 11us/sample - loss: 0.8409 - val_loss: 0.8454
Epoch 59/1000
51894/51894 [==============================] - 1s 11us/sample - loss: 0.8409 - val_loss: 0.8450
Epoch 60/1000
51894/51894 [==============================] - 1s 14us/sample - loss: 0.8408 - val_loss: 0.8448
Epoch 61/1000
51894/51894 [==============================] - 1s 12us/sample - loss: 0.8408 - val_loss: 0.8445
Epoch 62/1000
51894/51894 [==============================] - 1s 11us/sample - loss: 0.8408 - val_loss: 0.8451
Epoch 63/1000
51894/51894 [==============================] - 1s 17us/sample - loss: 0.8408 - val_loss: 0.8448
Epoch 64/1

Epoch 56/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8408 - val_loss: 0.8450
Epoch 57/1000
51895/51895 [==============================] - 1s 15us/sample - loss: 0.8408 - val_loss: 0.8449
Epoch 58/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8407 - val_loss: 0.8448
Epoch 59/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8407 - val_loss: 0.8454
Epoch 60/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8408 - val_loss: 0.8449
Epoch 61/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8407 - val_loss: 0.8457
Epoch 62/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8408 - val_loss: 0.8451
Epoch 63/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8408 - val_loss: 0.8457
Epoch 64/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8408 - val_loss: 0.8455
Epoch 65/1

51895/51895 [==============================] - 1s 12us/sample - loss: 0.8397 - val_loss: 0.8521
Epoch 19/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8398 - val_loss: 0.8516
Epoch 20/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8396 - val_loss: 0.8517
Epoch 21/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8398 - val_loss: 0.8519
Epoch 22/1000
51895/51895 [==============================] - 1s 15us/sample - loss: 0.8396 - val_loss: 0.8519
Epoch 23/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8396 - val_loss: 0.8515
Epoch 24/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8394 - val_loss: 0.8518
Epoch 25/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8395 - val_loss: 0.8512
Epoch 26/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8395 - val_loss: 0.8518
Epoch 27/1000
51895/5189

51895/51895 [==============================] - 1s 18us/sample - loss: 0.8388 - val_loss: 0.8510
Epoch 93/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8389 - val_loss: 0.8506
Epoch 94/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8389 - val_loss: 0.8508
Epoch 95/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8389 - val_loss: 0.8509
Epoch 96/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8388 - val_loss: 0.8512
Epoch 97/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8389 - val_loss: 0.8508
Epoch 98/1000
51895/51895 [==============================] - 1s 15us/sample - loss: 0.8388 - val_loss: 0.8508
Epoch 99/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8388 - val_loss: 0.8508
Epoch 100/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8389 - val_loss: 0.8509
Epoch 101/1000
51895/51

Epoch 55/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8456 - val_loss: 0.8301
Epoch 56/1000
51895/51895 [==============================] - 1s 12us/sample - loss: 0.8456 - val_loss: 0.8308
Epoch 57/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8457 - val_loss: 0.8306
Epoch 58/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8456 - val_loss: 0.8310
Epoch 59/1000
51895/51895 [==============================] - 1s 13us/sample - loss: 0.8457 - val_loss: 0.8308
Epoch 60/1000
51895/51895 [==============================] - 1s 11us/sample - loss: 0.8456 - val_loss: 0.8303
Epoch 61/1000
51895/51895 [==============================] - 1s 17us/sample - loss: 0.8457 - val_loss: 0.8306
Epoch 62/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8456 - val_loss: 0.8311
Epoch 63/1000
51895/51895 [==============================] - 1s 14us/sample - loss: 0.8456 - val_loss: 0.8309
Epoch 64/1

# Label weights

* Features and labels are encoded as dummy vectors (0 or 1)
* I'm using Keras to do a linear probability benchmark (with Lasso penalties)
* Main issue: some conditions are common while others are rare
* Why is this a problem: common conditions have higher variance and would dominate the Mean Square Error
* Solution: Define a custom loss that takes weights for features


In [517]:
%%writefile predictConditions.py
import getFeatures
import feature_weighted_mse
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.stats import t    
from statsmodels.stats.multitest import multipletests

Overwriting predictConditions.py


In [518]:
constant_full = pd.read_csv('constant_full.csv', index_col=0)
    
constant_full

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8
Female,0.000308,0.000166,0.000264,0.000288,0.000308,0.000309,0.000286,0.000244
Deceased,0.056281,0.074205,0.056274,0.049803,0.065216,0.050973,0.040013,0.057133
Age 0-4,0.000758,0.007184,-0.000243,-0.000186,0.000477,0.004553,0.001207,0.003928
Age 5-9,0.002589,0.007931,0.004926,0.000620,-0.003293,-0.006274,0.003479,-0.004181
Age 10-14,-0.003525,0.074625,-0.008108,0.004676,0.009623,-0.000126,0.000315,0.004645
...,...,...,...,...,...,...,...,...
"Localized, primary osteoarthritis of the hand",-0.000286,-0.000097,-0.000244,0.000259,-0.000232,0.001267,-0.000418,-0.000295
Osteoarthritis of hip,0.000749,-0.000209,0.000118,-0.000347,0.010369,0.000776,-0.000426,0.000154
Contact dermatitis,0.002194,0.000347,0.000281,0.000198,0.003015,0.000574,0.000282,-0.000201
Concussion injury of brain,0.000651,0.000167,-0.000390,0.000718,-0.000330,-0.000162,-0.000357,0.001793


In [519]:
coef_full = pd.read_csv('coef_full.csv', index_col=0)

coef_full

,Female,Age 0-4,Age 5-9,Age 10-14,Age 15-19,Age 20-24,Age 25-29,Age 30-34,Age 35-39,Age 40-44,...,Tubal pregnancy,Injury of anterior cruciate ligament,Overlapping malignant neoplasm of colon,Malignant tumor of colon,Chronic obstructive bronchitis (disorder),Seasonal allergic rhinitis,Pulmonary emphysema (disorder),Suspected lung cancer (situation),Non-small cell lung cancer (disorder),"Non-small cell carcinoma of lung, TNM stage 1 (disorder)"
Female,"[0.9994238615036011, 0.9994780421257019, 0.999...","[0.0001972150639630854, -0.0001622617419343441...","[1.843123754952103e-05, 0.00015203049406409264...","[3.70907764590811e-05, 0.00022644212003797293,...","[8.786928083281964e-05, 0.00017806136747822165...","[5.59463442186825e-05, -3.954165367758833e-05,...","[7.175697828643024e-05, 0.00018811860354617238...","[0.00022584674297831953, -0.000144424717291258...","[-2.543208665883867e-06, 0.0001201063860207796...","[0.00030227735987864435, -3.263737016823143e-0...",...,"[6.230683356989175e-05, 0.00017113138164859265...","[8.534589142072946e-05, 2.2143518435768783e-05...","[3.0813360353931785e-05, 0.0001030616258503869...","[3.618368646129966e-05, -3.6144774639979005e-0...","[-5.5237600463442504e-05, -0.00020721020700875...","[-9.439473797101527e-05, -0.000240332490648143...","[4.4941873056814075e-05, 0.0001336292625637725...","[2.8802787710446864e-05, 4.975481715518981e-06...","[1.5613528375979513e-05, 0.0002526500029489398...","[0.00027655003941617906, 4.7199137043207884e-0..."
Deceased,"[-0.0073789022862911224, -0.012394324876368046...","[-0.04644082486629486, -0.05925685167312622, -...","[-0.046548955142498016, -0.0644829049706459, -...","[-0.04596250504255295, -0.06376709789037704, -...","[-0.04789375886321068, -0.06498119235038757, -...","[-0.04321202263236046, -0.061850663274526596, ...","[-0.03989005461335182, -0.06104419752955437, -...","[-0.0453440360724926, -0.06076515093445778, -0...","[-0.03650955110788345, -0.06314641237258911, -...","[-0.04088449478149414, -0.05222180113196373, -...",...,"[-3.854333044728264e-05, -4.0612816519569606e-...","[-0.003969011828303337, -5.996434629196301e-05...","[-0.004762034863233566, -0.005148023366928101,...","[0.03961325064301491, 0.0869751125574112, 0.00...","[-0.001616183202713728, 0.01365602295845747, 0...","[-0.00025207287399098277, -0.00021494799875654...","[0.054832253605127335, 0.05059337615966797, -0...","[0.00010571414168225601, 0.008106790482997894,...","[-0.0004772221145685762, 0.009110445156693459,...","[-0.00043111812556162477, 0.009283681400120258..."
Age 0-4,"[0.0010205016005784273, 0.0007051500724628568,...","[0.8044350147247314, 0.8107116222381592, 0.800...","[-0.0012565191136673093, -0.007942652329802513...","[-0.00014739990001544356, -0.00813709292560815...","[-0.00156314333435148, -0.006744459271430969, ...","[-0.0010858167661353946, -0.007710026111453772...","[-0.0017585817258805037, -0.007662032265216112...","[-0.0015511447563767433, -0.007595157716423273...","[-0.0009803103748708963, -0.00642688013613224,...","[-0.0008303343201987445, -0.006820457056164741...",...,"[-0.00010418239253340289, -8.746171806706116e-...","[-0.00012988319213036448, -0.00012213023728691...","[1.5436758985742927e-05, 5.116549436934292e-05...","[-6.384904554579407e-05, -0.000182560746907256...","[0.00020524815772660077, -3.196402394678444e-0...","[-0.005745735019445419, 0.0030566726345568895,...","[9.111770486924797e-05, -4.3333740904927254e-0...","[2.0044150005560368e-05, -7.79762485763058e-05...","[-2.4963184841908514e-05, 9.122402843786404e-0...","[7.959279173519462e-05, 2.37040949286893e-05, ..."
Age 5-9,"[-0.0001450422714697197, -0.003817510325461626...","[0.18661227822303772, 0.16298121213912964, 0.1...","[0.790812075138092, 0.772051215171814, 0.78967...","[-0.0021195514127612114, -0.006445011589676142...","[-0.001565655809827149, -0.008606037124991417,...","[-0.0013227805029600859, -0.006895039230585098...","[-0.00283366022631526, -0.006881819572299719, ...","[-0.0026112745981663465,

In [521]:
constant_name = pd.Series(constant_full.index, index=constant_full.index, name='name')

constant_mean = constant_full.mean(axis=1)
constant_mean.name = 'mean'

constant_std = constant_full.std(axis=1, ddof=1)
constant_std.name = 'std'

constant_df = pd.concat(
    [constant_name, constant_mean, constant_std], axis=1)

constant_df['p-value'] = (1 - t.cdf(
    x=abs(constant_mean/constant_std), df=n_splits*n_repeats-1)) * 2

constant_df['FDR adj p-value'] = multipletests(constant_df['p-value'], method='fdr_bh')[1]

constant_df.to_csv('constant.csv', index=False)

coef_mean = coef_full.applymap(lambda x: np.mean(json.loads(x)))
coef_mean.to_csv('coef_mean.csv')

coef_std = coef_full.applymap(lambda x: np.std(json.loads(x), ddof=1))
coef_std.to_csv('coef_std.csv')

coef_p_value = pd.DataFrame(
    (1 - t.cdf(x=abs(coef_mean/coef_std), df=n_splits*n_repeats-1)) * 2,
    index=coef_full.index,
    columns=coef_full.columns
)
coef_p_value.to_csv('coef_p_value.csv')

coef_p_value_adj = multipletests(
    coef_p_value.values.reshape((-1,)), 
    method='fdr_bh')[1].reshape(coef_p_value.shape)

coef_p_value_adj = pd.DataFrame(
    coef_p_value_adj, index=y_train_df.columns, columns=x_train_df.columns)
coef_p_value_adj.to_csv('coef_p_value_adj.csv')

In [523]:
# constant_df = pd.read_csv('constant.csv', index_col=0)
# constant_df
# coef_mean = pd.read_csv('coef_mean.csv', index_col=0)
# coef_mean
# coef_std = pd.read_csv('coef_std.csv', index_col=0)
# coef_std
# coef_p_value = pd.read_csv('coef_p_value.csv', index_col=0)
# coef_p_value.head(50).iloc[:,:20]
# coef_p_value_adj = pd.read_csv('coef_p_value_adj.csv', index_col=0)
# coef_p_value_adj

print('Number of tests: ', coef_mean.shape[0]*coef_mean.shape[1])

Number of tests:  15678


In [524]:
print('Regular p-value %5 rejection rate', (np.abs(coef_p_value.values) < 0.01).mean())
print('Adjusted p-value %5 rejection rate', (np.abs(coef_p_value_adj.values) < 0.01).mean())

Regular p-value %5 rejection rate 0.022962112514351322
Adjusted p-value %5 rejection rate 0.006569715524939406


In [525]:
nonzero = coef_p_value.applymap(lambda x: x <= 0.01).values.nonzero()
nonzero[0].shape, nonzero[1].shape 

((360,), (360,))

In [529]:
tuples = [(
    coef_mean.index[i], 
    coef_mean.columns[j], 
    coef_mean.iloc[i,j],
    coef_std.iloc[i,j],
    coef_p_value.iloc[i,j],
    coef_p_value_adj.iloc[i,j],
) for i,j in zip(nonzero[0],nonzero[1])]

coef_df = pd.DataFrame(tuples, columns=['y','x','mean', 'std', 'p-value', 'FDR adj p-value'])
coef_df.to_csv('coef.csv', index=False)
coef_df

,y,x,mean,std,p-value,FDR adj p-value
0,Female,Female,0.999540,0.000068,0.000000,0.000000
1,Deceased,Age 0-4,-0.045816,0.009080,0.001487,0.107157
2,Deceased,Age 5-9,-0.049527,0.008855,0.000822,0.069650
3,Deceased,Age 10-14,-0.049112,0.008841,0.000855,0.071693
4,Deceased,Age 15-19,-0.049202,0.008364,0.000610,0.055868
...,...,...,...,...,...,...
355,Chronic intractable migraine without aura,Impacted molars,0.107712,0.020580,0.001207,0.092779
356,Perennial allergic rhinitis,Childhood asthma,0.041448,0.008088,0.001362,0.099788
357,Hyperglycemia (disorder),Diabetes,0.046885,0.009537,0.001721,0.119936
358,Diabetic renal disease (disorder),Neuropathy due to type 2 diabetes mellitus (di...,0.066444,0.016437,0.004919,0.267770
